# OD Pair based analysis

In [15]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.float_format = '{:,.2f}'.format

## read data
* od matrices: Prt, PuT

In [16]:
od_PuT = pd.read_csv('data//MTX_203.csv')
del od_PuT['Unnamed: 0']
od_PuT.columns = [int(c) for c in od_PuT.columns]
od_PrT = pd.read_csv('data//MTX_204.csv')
del od_PrT['Unnamed: 0']
od_PrT.columns = [int(c) for c in od_PrT.columns]

In [17]:
od_PrT = od_PrT.stack().reset_index()
od_PrT.columns=['Z_Rejon',"Do_Rejon", 'TripsPrT']
od_PuT = od_PuT.stack().reset_index()
od_PuT.columns=['Z_Rejon',"Do_Rejon", 'TripsPuT']
od = pd.merge(od_PrT, od_PuT, on = ['Z_Rejon', "Do_Rejon"])

* trips

In [18]:
trips = pd.read_csv('data//From_Via_To.csv')
del trips['Unnamed: 0']

## prepare

In [19]:
trips.Czas_PrT = trips.Czas_PrT/60 # przeliczam na minuty
trips.Czas_PuT = trips.Czas_PuT/60 *0.5 # tu zakladam dwa razy krotsze, dla obliczen, w Warszawie tak nie bedzie
BUDGET_PRT = trips['Czas_PrT'].quantile(0.5) # przyjmuje ze liczy sie tylko dolna polowa czasow
BUDGET_PUT = BUDGET_PRT #trips['Czas_PuT'].quantile(0.5) # a tu juz zakladam ze budzety sa rowne
trips['time_left_PrT']=BUDGET_PRT - trips.Czas_PrT # ile czasu zostaje ponizej budzetu w POI
trips['time_left_PuT']=BUDGET_PUT - trips.Czas_PuT
trips.Z_Rejon = trips.Z_Rejon.astype(int)
trips.Do_Rejon = trips.Do_Rejon.astype(int)
pd.options.display.float_format = '{:,.1f}'.format

In [20]:
trips = pd.merge(trips,od, on = ['Z_Rejon',"Do_Rejon"])
trips['odPair'] = trips["Z_Rejon"].astype(int)*1000+trips["Do_Rejon"].astype(int)

# OD pairs-based analysis:
## number of POI available
- total minutes
- mean/median minutes

In [21]:
print('number of POIs within budget per OD Pair - PuT/PrT')
nPOIs = trips[trips.Czas_PrT<BUDGET_PRT].groupby(by=['Z_Rejon',"Do_Rejon"])['POI'].nunique().to_frame('nPOIsPRT')
nPOIs['nPOIsPUT'] = trips[trips.Czas_PuT<BUDGET_PUT].groupby(by=['Z_Rejon',"Do_Rejon"])['POI'].nunique()
nPOIs.reset_index().head()

number of POIs within budget per OD Pair - PuT/PrT


,Z_Rejon,Do_Rejon,nPOIsPRT,nPOIsPUT
0,1,1,21,1.0
1,1,2,26,1.0
2,1,3,26,nan
3,1,4,26,nan
4,1,5,20,nan


## mean/median time available

In [22]:
print("time per OD pair  PrT")
t = trips[trips.Czas_PrT<BUDGET_PRT].groupby(
    by=['Z_Rejon',"Do_Rejon"])['Czas_PrT', 'time_left_PrT'].agg(['sum','mean','median']).reset_index()
t.head()

time per OD pair  PrT


Z_Rejon Do_Rejon Czas_PrT             time_left_PrT            
                        sum mean median           sum mean median
0       1        1    166.6  7.9    7.9          83.2  4.0    4.0
1       1        2    224.7  8.6    9.0          84.5  3.3    2.9
2       1        3    227.8  8.8    9.1          81.5  3.1    2.8
3       1        4    242.7  9.3    9.2          66.5  2.6    2.7
4       1        5    183.4  9.2    9.0          54.5  2.7    2.9

In [23]:
print("time per OD pair PuT")
t = trips[trips.Czas_PuT<BUDGET_PUT].groupby(
    by=['Z_Rejon',"Do_Rejon"])['Czas_PuT', 'time_left_PuT'].agg(['sum','mean','median']).reset_index()
t.head()

time per OD pair PuT


Z_Rejon Do_Rejon Czas_PuT             time_left_PuT            
                        sum mean median           sum mean median
0       1        1     11.5 11.5   11.5           0.4  0.4    0.4
1       1        2     11.8 11.8   11.8           0.1  0.1    0.1
2       1       10     33.5 11.2   11.3           2.1  0.7    0.6
3       1       13     11.8 11.8   11.8           0.1  0.1    0.1
4       1       15     56.3 11.3   11.4           3.2  0.6    0.5